Import all packages here

In [ ]:
import tensorflow as tf
from transformers import Wav2Vec2Processor, TFWav2Vec2Model

import librosa
from librosa.effects import trim

import soundfile

import scipy as sy
import pydub as pyd

import jiwer as jw

import pandas as pd 
import numpy as np 

import os

c:\Users\Marwin Jay\Desktop\ASR Client\asr-thesis-2024\asr_env\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Mounting Gdrive for the datasets

Setting up the Dataset

In [ ]:
dataset_path = "dataset/"
metatadata_path = "metadata"

audio_directory = "dir"

audio_data = []

# Create a dataframe for the metadata
dataframe = pd.read_csv(metatadata_path)

# Display dataframe to check if ready

print(dataframe.head())






Data Pre-Processing

In [ ]:
# Load the audio files and preprocess

for filename in os.listdir(audio_directory):
    if(filename.endswith(".wav")):
        file_path = os.path.join(audio_directory, filename)
        
        # Load file with Sampling Rate of 16000 Hz
        signal, sr = librosa.load(file_path, sr=16000)
        
        # Normalizing the audio data
        signal = signal / max(abs(signal))
        
        # Trim silence
        signal, _ = trim(signal)
        
        # Appending the data to a variable
        audio_data.append({"file_path": file_path, "signal": signal, "Sampling Rate": sr })


# Check if audio_data is ready

print(f"File: {audio_data[0]['file_path']}, Signal Length: {audio_data[0]['signal']}")

Training the Model

Training Results

Visualization of Results using MatPlotLib